### Установка (при необходимости раскомментить)

In [1]:
# # Uninstall previous version of WhiteBox
# !pip3 uninstall --yes autowoe 
# # install requirements if Linux 
# # !pip3 install -r requirements.txt
# # Install WhiteBox
# !python3 setup.py install --user

In [2]:
import pandas as pd
import numpy as np
import lightgbm as lgb

from collections import OrderedDict
from sklearn.metrics import roc_auc_score
from copy import deepcopy

from autowoe import ReportDeco, AutoWoE

### Чтение и подготовка обучающей выборки

In [3]:
train = pd.read_csv("./example_data/train_demo.csv",
                    low_memory=False,
                    index_col="line_id",
                    parse_dates = ["datetime_" + str(i) for i in range(2)],)

### Чтение и подготовка тестовой выборки

In [4]:
test = pd.read_csv("./example_data/test_demo.csv",
                   index_col="line_id", 
                   parse_dates = ["datetime_" + str(i) for i in range(2)])
    
test_target = pd.read_csv("./example_data/test-target_demo.csv")["target"]
test["target"] = test_target.values

### Параметры модели

Для обучения модели рекомендуется указать тип признаков для обучения.
Поэтому создается словарь features_type с ключами: 

 "real" -- вещественный признак
 
 "cat" --  категориальный.
 
 __"date"-- ("%Y%d%m", ("m", "d", "wd", "h", "min"))__
 
 Для признаков, которые не размечены, типы будут определены автоматом. Такой вариант будет работать, но качество порядочно просядет
 
__Попробуем указать даты с форматом None (автоопределение) и сезонностью - день месяца и день недели__

#### features_type

In [5]:
num_col = list(filter(lambda x: "numb" in x, train.columns))
num_feature_type = {x: "real" for x in num_col}

date_col = list(filter(lambda x: "datetime" in x, train.columns))
date_feature_type = {x: (None, ("d", "wd")) for x in date_col}

In [6]:
features_type = dict(**num_feature_type,
                     **date_feature_type
                    )

In [7]:
# подробно параметры описаны в Example_1
auto_woe = AutoWoE(monotonic=True,
                     max_bin_count=4,
                     oof_woe=False,
                     regularized_refit=False,
                     p_val=0.05,
                     debug=False
        )
auto_woe = ReportDeco(auto_woe)

In [8]:
auto_woe.fit(train[num_col + date_col + ['target']], 
             target_name="target",
             features_type=features_type,
            )

[autowoe.lib.selectors.selector_first] [INFO]  features ['number_16', 'number_17', 'number_18', 'number_19', 'number_20', 'number_21', 'number_22', 'number_23', 'number_24', 'number_25', 'number_26', 'number_27', 'number_28', 'number_30', 'number_31', 'number_32', 'number_34', 'number_35', 'number_36', 'number_37', 'number_40', 'number_41', 'number_42', 'number_96', 'number_97', 'number_99', 'number_100', 'number_101', 'number_105', 'number_113', 'number_115', 'number_116', 'number_117', 'number_118', 'number_119', 'number_120', 'number_121', 'number_122', 'number_123', 'number_124', 'number_125', 'number_126', 'number_127', 'number_128', 'number_129', 'number_131', 'number_132', 'number_133', 'number_134', 'number_135', 'number_141', 'number_142', 'number_143', 'number_144', 'number_145', 'number_147', 'number_149', 'number_150', 'number_151', 'number_152', 'number_208', 'number_209', 'number_210', 'number_211', 'number_212', 'number_213', 'number_214', 'number_215', 'number_216', 'nu

/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:1291: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[9]	val_set's auc: 0.716979
[autowoe.lib.selectors.selector_first] [INFO]  features ['number_0', 'number_2', 'number_3', 'number_5', 'number_6', 'number_7', 'number_10', 'number_11', 'number_14', 'number_15', 'number_29', 'number_33', 'number_38', 'number_39', 'number_43', 'number_44', 'number_45', 'number_46', 'number_47', 'number_48', 'number_49', 'number_50', 'number_51', 'number_52', 'number_53', 'number_54', 'number_55', 'number_56', 'number_57', 'number_58', 'number_59', 'number_60', 'number_61', 'number_62', 'number_63', 'number_64', 'number_65', 'number_66', 'number_67', 'number_68', 'number_69', 'number_70', 'number_71', 'number_72', 'number_73', 'number_74', 'number_75', 'number_76', 'number_77', 'number_78', 'number_79', 'number_80', 'number_81', 'number_82', 'number_83', 'number_84', 'number_85', 'number_86', 'number_87', 'number_88', 'number_89', 'number_90', 'number_91', 'numbe

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=6.
  % (min_groups, self.n_splits)), UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=6.
  % (min_groups, self.n_splits)), UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=6.
  % (min_groups, self.n_splits)), UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=6.
  % (min_groups, self.n_splits)), UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in 

[autowoe.lib.autowoe] [INFO] number_707 processing...
[autowoe.lib.autowoe] [INFO] number_758 processing...

...

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=6.
  % (min_groups, self.n_splits)), UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=6.
  % (min_groups, self.n_splits)), UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=6.
  % (min_groups, self.n_splits)), UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=6.
  % (min_groups, self.n_splits)), UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in 

[autowoe.lib.autowoe] [INFO] number_763 processing...


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=6.
  % (min_groups, self.n_splits)), UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=6.
  % (min_groups, self.n_splits)), UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=6.
  % (min_groups, self.n_splits)), UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=6.
  % (min_groups, self.n_splits)), UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in 

[autowoe.lib.autowoe] [INFO] datetime_0__F__d processing...
[autowoe.lib.autowoe] [INFO] number_764 processing...
[autowoe.lib.autowoe] [INFO] number_577 processing...
[autowoe.lib.autowoe] [INFO] datetime_1__F__wd processing...
[autowoe.lib.autowoe] [INFO] number_749 processing...
[autowoe.lib.autowoe] [INFO] number_743 processing...
[autowoe.lib.autowoe] [INFO] number_754 processing...
[autowoe.lib.autowoe] [INFO] number_759 processing...
[autowoe.lib.autowoe] [INFO] number_739 processing...
[autowoe.lib.autowoe] [INFO] datetime_0__F__wd processing...
[autowoe.lib.autowoe] [INFO] datetime_1__F__d processing...
[autowoe.lib.autowoe] [INFO] dict_keys(['number_1', 'number_4', 'number_8', 'number_9', 'number_12', 'number_13', 'number_102', 'number_103', 'number_139', 'number_154', 'number_155', 'number_157', 'number_158', 'number_160', 'number_162', 'number_164', 'number_165', 'number_166', 'number_167', 'number_170', 'number_171', 'number_173', 'number_175', 'number_176', 'number_178', 

[autowoe.lib.utilities.refit] [INFO] Iter 0 of final refit starts with 19 features
[autowoe.lib.utilities.refit] [INFO] Iter 1 of final refit starts with 18 features
[autowoe.lib.utilities.refit] [INFO] Iter 2 of final refit starts with 17 features
[autowoe.lib.utilities.refit] [INFO] Iter 3 of final refit starts with 16 features
[autowoe.lib.utilities.refit] [INFO] Iter 4 of final refit starts with 15 features
[autowoe.lib.utilities.refit] [INFO] Iter 5 of final refit starts with 14 features
[autowoe.lib.utilities.refit] [INFO] Iter 6 of final refit starts with 13 features
[autowoe.lib.utilities.refit] [INFO] Iter 7 of final refit starts with 12 features
[autowoe.lib.utilities.refit] [INFO] Iter 8 of final refit starts with 11 features
[autowoe.lib.utilities.refit] [INFO] Iter 9 of final refit starts with 10 features
[autowoe.lib.utilities.refit] [INFO] Iter 10 of final refit starts with 9 features
[autowoe.lib.autowoe] [INFO] number_254         -0.494361
number_754         -0.471157


In [9]:

pred = auto_woe.predict_proba(test)
roc_auc_score(test['target'], pred)

0.7729983249581239

##### Замечание
ReportDeco - обертка для построения отчета. Она не обязательна для обучения и применения модели, но обязательна для построения отчета (см последнюю ячейку).

### Значения коэфициентов и p-values

При указании regularized_refit=False будет произведена оценка p-value на коэфициенты модели. Коэфициенты с p-value выше указанного порога не будут включены в модель

In [10]:
auto_woe.features_fit

number_254         -0.494361
number_754         -0.471157
number_345         -0.760179
number_759         -0.758034
number_761         -0.882112
number_706         -0.685612
number_1           -1.134868
number_368         -1.050152
datetime_1__F__d   -1.237848
dtype: float64

In [11]:
auto_woe.intercept

-4.536591143939794

In [12]:
auto_woe.p_vals

number_254          0.011516
number_754          0.021124
number_345          0.002208
number_759          0.001213
number_761          0.000470
number_706          0.004311
number_1            0.000459
number_368          0.000007
datetime_1__F__d    0.003885
Intercept_          0.000000
dtype: float64

### Формирование отчета

In [13]:
report_params = {"automl_date_column": "report_month", # колонка с датой в формате params['datetimeFormat']
                 "output_path": "./AUTOWOE_REPORT_2", # папка, куда сгенерится отчет и сложатся нужные файлы
                 "report_name": "___НАЗВАНИЕ ОТЧЕТА___",
                 "report_version_id": 1,
                 "city": "Воронеж",
                 "model_aim": "___ЦЕЛЬ ПОСТРОЕНИЯ МОДЕЛИ___",
                 "model_name": "___НАЗВАНИЕ МОДЕЛИ___",
                 "zakazchik": "___ЗАКАЗЧИК___",
                 "high_level_department": "___ПОДРАЗДЕЛЕНИЕ___",
                 "ds_name": "___РАЗРАБОТЧИК МОДЕЛИ___",
                 "target_descr": "___ОПИСАНИЕ ЦЕЛЕВОГО СОБЫТИЯ___",
                 "non_target_descr": "___ОПИСАНИЕ НЕЦЕЛЕВОГО СОБЫТИЯ___"}

auto_woe.generate_report(report_params)

No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
/mnt/alex/Whitebox_git/autowoe/lib/report/utilities_images/utilities_images.py:186: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  grp = df.groupby(col)['pred', 'Target'].mean()
/mnt/alex/Whitebox_git/autowoe/lib/report/utilities_images/utilities_images.py:186: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  grp = df.groupby(col)['pred', 'Target'].mean()
/mnt/alex/Whit

[autowoe.lib.report.report_generator] [INFO] Successfully wrote ./AUTOWOE_REPORT_2/autowoe_report.html.
